# Neural Networks

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [3]:
net = Net()
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [5]:
params[0]

Parameter containing:
tensor([[[[-0.2210, -0.2765, -0.0455],
          [ 0.0864, -0.1285, -0.0930],
          [-0.2322,  0.3014, -0.2835]]],


        [[[ 0.3064,  0.0650, -0.2679],
          [ 0.0945,  0.0995,  0.0266],
          [ 0.1780,  0.2078, -0.1206]]],


        [[[ 0.2344, -0.0504, -0.0497],
          [-0.0893, -0.3302, -0.1080],
          [ 0.1611, -0.1800, -0.2713]]],


        [[[ 0.1090,  0.3172, -0.0878],
          [ 0.2506, -0.0345, -0.2155],
          [-0.0749,  0.1038, -0.1054]]],


        [[[-0.3133,  0.0095, -0.2337],
          [ 0.1707,  0.2183,  0.0878],
          [-0.1358, -0.2574, -0.3303]]],


        [[[-0.0798,  0.0019,  0.3150],
          [ 0.0800,  0.2885, -0.1270],
          [ 0.1937, -0.1662,  0.3192]]]], requires_grad=True)

In [6]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1055,  0.0048, -0.0426, -0.0378, -0.0480, -0.0836, -0.0643,  0.0608,
          0.0399,  0.0160]], grad_fn=<AddmmBackward>)


In [7]:
net.zero_grad()
out.backward(torch.rand(1, 10))

## Loss Function

In [8]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.5068, grad_fn=<MseLossBackward>)


In [9]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## Backprop

In [10]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0053,  0.0131, -0.0039, -0.0034,  0.0017,  0.0059])


## Update weights

### SGD

In [11]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [12]:
import torch.optim as optim

# Optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# Training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update